In [ ]:
import face_recognition

image = face_recognition.load_image_file("../data/face_recognition_images/person2.1.jpg")

unknown_image = face_recognition.load_image_file("/home/vorkov/Workspace/EDA/inference/image.jpeg")

image_encoding = face_recognition.face_encodings(image)[0]
unknown_encoding = face_recognition.face_encodings(unknown_image)[0]
print("image type: ", unknown_image)

results = face_recognition.compare_faces([image_encoding], unknown_encoding, tolerance=0.66)
results

In [ ]:
import datetime
import face_recognition
import cv2

identified_face_encodings = []
identified_face_names = []

name = "Kostylev Ivan"
learned_person = face_recognition.load_image_file("../data/face_recognition_images/person1.1.jpg")
face_locations = face_recognition.face_locations(learned_person)
learned_person_encoding = face_recognition.face_encodings(learned_person, face_locations)[0]
identified_face_encodings.append(learned_person_encoding)
identified_face_names.append(name)

name = "Vorkov Nikita"
learned_person = face_recognition.load_image_file("../data/face_recognition_images/person2.1.jpg")
face_locations = face_recognition.face_locations(learned_person)
learned_person_encoding = face_recognition.face_encodings(learned_person, face_locations)[0]
identified_face_encodings.append(learned_person_encoding)
identified_face_names.append(name)

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []

gst_stream = 'rtspsrc location=rtsp://127.0.0.1:18554/test user-id=user user-pw=pass latency=0 ! rtpjitterbuffer drop-on-latency=true ! decodebin ! videoconvert ! appsink'
cap = cv2.VideoCapture(gst_stream, cv2.CAP_GSTREAMER)
while True:
    start = datetime.datetime.now()
    success, frame = cap.read()

    # Only process every other frame of video to save time
    if success:
        # small_frame = cv2.resize(frame, (48, 48), interpolation = cv2.INTER_LINEAR)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        small_frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
        rgb_iamge = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_iamge)
        face_encodings = face_recognition.face_encodings(rgb_iamge, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            matches = face_recognition.compare_faces(identified_face_encodings, face_encoding)
            name = "Unknown"

            # Если хотим брать только идентифицированные лица
            if True in matches:
                first_match_index = matches.index(True)
                name = identified_face_names[first_match_index]
                face_names.append(name)
                
            # Если хотим брать лица близкие к изученным            
            # face_distances = face_recognition.face_distance(identified_face_encodings, face_encoding)
            # best_match_index = np.argmin(face_distances)
            # if matches[best_match_index]:
            #     name = identified_face_names[best_match_index]
            # face_names.append(identified_face_encodings)

    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(small_frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    end = datetime.datetime.now()
    fps = f"FPS: {1 / (end - start).total_seconds():.2f}"
    cv2.putText(small_frame, fps, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 8)
    
    cv2.imshow('Video', small_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()